In [2]:
import tensorflow as tf
import tensorflow.keras as keras
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np

%matplotlib inline

print("Tensorflow Version: {}".format(tf.__version__))
print("Tensorflow.Keras Version: {}".format(keras.__version__))

Tensorflow Version: 1.13.2
Tensorflow.Keras Version: 2.2.4-tf


In [ ]:
pb_path = "/content/output_graph.pb"
inputs = ["Placeholder"]
outputs = ["final_result"]

In [ ]:
%pwd

In [ ]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(pb_path, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

def show_operation_names(count=10):
  """
  Show all operation names in the graph, including "input(image_tensor)"
  and "output(detection_boxes, detection_scores, detection_classes, num_detections)".
  """
  with detection_graph.as_default():
    with tf.Session() as sess:
      opts = tf.get_default_graph().get_operations()
      for opt in opts[:count]:
        for output in opt.outputs: print(output.name)
      print("...")
      for opt in opts[-count:]:
        for output in opt.outputs: print(output.name)

show_operation_names(10)

In [ ]:
%pwd

In [ ]:
with tf.Session(graph=detection_graph) as sess:
  output_graph_def = tf.graph_util.convert_variables_to_constants(
        sess, detection_graph.as_graph_def(), ["final_result"])
  
  with tf.gfile.FastGFile("tf1.14.pb", 'wb') as fout:
      fout.write(output_graph_def.SerializeToString())

In [ ]:
with tf.Graph().as_default():
  inputs = detection_graph.get_tensor_by_name("Placeholder:0")
  outputs = detection_graph.get_tensor_by_name("final_result:0")
  with tf.Session() as sess:
    converter = tf.lite.TFLiteConverter.from_session(sess, inputs, outputs)
    tflite_model = converter.convert()
    open("converted_model.tflite", "wb").write(tflite_model)

In [ ]:
tf.reset_default_graph()

converter = tf.lite.TFLiteConverter.from_frozen_graph("tf1.14.pb", ["Placeholder"], ["final_result"])
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)